In [119]:
# Install the igraph library

library(igraph)

In [120]:

# Helper function for getting warcbase data into igraph
processFile = function(filepath) {
  con = file(filepath, "r")
  line = readLines(con)
  line <- as.data.frame(do.call(rbind, strsplit(gsub('\\(|\\)', '', line), ',')), stringsAsFactors=FALSE)
  names(line) = c("date", "from", "to", "weight")
  return (line)
  close(con)
}


In [117]:

# 
filepath = '~/data/links/'
file = 'ALBERTA_canadian_business_grey_literature'
suffix = '-links.txt'

savesuffix = '.graphml'
savepath = '~/data/graphml/'

filename = paste0(filepath, file, suffix)
savename = paste0(savepath, file, savesuffix)

graph_file <- graph.data.frame(processFile(filename)[c("from", "to", "weight")], directed=TRUE)

In [118]:
write.graph (graph_file, savename, format='graphml')

In [116]:
graph_file

IGRAPH DNW- 142 470 -- 
+ attr: name (v/c), weight (e/c)
+ edges (vertex names):
 [1] ceocouncil.ca     ->ceocouncil.ca collierscanada.com->tbe.taleo.net
 [3] collierscanada.com->tbe.taleo.net collierscanada.com->tbe.taleo.net
 [5] collierscanada.com->tbe.taleo.net collierscanada.com->tbe.taleo.net
 [7] collierscanada.com->youtube.com   collierscanada.com->twitter.com  
 [9] collierscanada.com->colliers.com  collierscanada.com->linkedin.com 
[11] collierscanada.com->youtube.com   collierscanada.com->colliers.com 
[13] collierscanada.com->linkedin.com  collierscanada.com->twitter.com  
[15] collierscanada.com->twitter.com   collierscanada.com->youtube.com  
+ ... omitted several edges